In [1]:
# Standard library
import json
from pathlib import Path
import joblib
import numpy as np
import pandas as pd
from scipy.signal import find_peaks


# Scikit-learn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, make_scorer, fbeta_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight

# XGBoost
from xgboost import XGBClassifier

# TensorFlow / Keras
from tensorflow import keras
from tensorflow.keras import layers

2026-01-15 21:18:19.772565: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-15 21:18:19.773121: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-15 21:18:19.819923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-15 21:18:21.216757: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [2]:
# Base directory
path_dir = Path.cwd()

# Folder with the JSON files
json_dir = path_dir / "per_point_v2"

# Prepare an empty DataFrame with the expected columns and an index name
df = pd.DataFrame(columns=["x", "y", "visible", "action"])
df.index.name = "image_frame"

frames_df = []

for json_path in json_dir.glob("*.json"):
    with json_path.open("r", encoding="utf-8") as f:
        ball_data = json.load(f)  # expected: dict keyed by image_frame

    # Build a DataFrame from the JSON dict, then transpose:
    file_df = pd.DataFrame(ball_data).T
    file_df.index.name = "image_frame"

    # Ensure column names match the expected schema
    file_df = file_df.reindex(columns=["x", "y", "visible", "action"])

    frames_df.append(file_df)

# Final concatenation
df = pd.concat(frames_df, axis=0, ignore_index=False)
df.index.name = "image_frame"

In [8]:
def build_features(
    df: pd.DataFrame,
    smooth_window: int = 7,
) -> pd.DataFrame:
    """
    Feature builder for ball hit / bounce detection.

    """

    # ------------------------------------------------------------------
    # Numeric positions and index
    # ------------------------------------------------------------------
    new_df = df.copy()
    new_df.index = pd.to_numeric(new_df.index, errors="coerce")
    new_df = new_df.sort_index()
    new_df["x_i"] = pd.to_numeric(new_df["x"], errors="coerce")
    new_df["y_i"] = pd.to_numeric(new_df["y"], errors="coerce")
    new_df = new_df.dropna(subset=["x_i", "y_i"])
    

    # ------------------------------------------------------------------
    # Raw positions
    # ------------------------------------------------------------------
    new_df["x_raw"] = new_df["x_i"]
    new_df["y_raw"] = new_df["y_i"]

    # ------------------------------------------------------------------
    # Centered smoothing on positions
    # ------------------------------------------------------------------

    # Centered rolling mean reduces high-frequency measurement noise
    # without eliminating physical discontinuities (hits / bounces).
    new_df["x_smooth"] = (
        new_df["x_raw"]
        .rolling(smooth_window, center=True, min_periods=1)
        .mean()
    )
    new_df["y_smooth"] = (
        new_df["y_raw"]
        .rolling(smooth_window, center=True, min_periods=1)
        .mean()
    )

    # ------------------------------------------------------------------
    # Time step (central)
    # ------------------------------------------------------------------
    t = new_df.index.to_series()

    # ------------------------------------------------------------------
    # Smoothed derivatives (stable kinematics)
    # ------------------------------------------------------------------
    x_smooth = new_df["x_smooth"].to_numpy()
    y_smooth = new_df["y_smooth"].to_numpy()

    vx = np.gradient(x_smooth, t)
    vy = np.gradient(y_smooth, t)

    ax = np.gradient(vx, t)
    ay = np.gradient(vy, t)

    jx = np.gradient(ax, t)
    jy = np.gradient(ay, t)

    new_df["vx"] = vx
    new_df["vy"] = vy
    new_df["ax"] = ax
    new_df["ay"] = ay
    new_df["jx"] = jx
    new_df["jy"] = jy

    # ------------------------------------------------------------------
    # Raw derivatives (impulse-sensitive)
    # ------------------------------------------------------------------
    x_raw = new_df["x_raw"].to_numpy()
    y_raw = new_df["y_raw"].to_numpy()

    vx_raw = np.gradient(x_raw, t)
    vy_raw = np.gradient(y_raw, t)

    ax_raw = np.gradient(vx_raw, t)
    ay_raw = np.gradient(vy_raw, t)

    jx_raw = np.gradient(ax_raw, t)
    jy_raw = np.gradient(ay_raw, t)

    new_df["vx_raw"] = vx_raw
    new_df["vy_raw"] = vy_raw
    new_df["ax_raw"] = ax_raw
    new_df["ay_raw"] = ay_raw
    new_df["jx_raw"] = jx_raw
    new_df["jy_raw"] = jy_raw

    # ------------------------------------------------------------------
    # Raw derivatubes in absolute
    # ------------------------------------------------------------------

    new_df["vx_abs_raw"] = np.abs(new_df["vx_raw"])
    new_df["vy_abs_raw"] = np.abs(new_df["vy_raw"])
    new_df["ax_abs_raw"] = np.abs(new_df["ax_raw"])
    new_df["ay_abs_raw"] = np.abs(new_df["ay_raw"])
    new_df["jx_abs_raw"] = np.abs(new_df["jx_raw"])
    new_df["jy_abs_raw"] = np.abs(new_df["jy_raw"])

    # ------------------------------------------------------------------
    # Magnitudes (smoothed)
    # ------------------------------------------------------------------
    new_df["v"] = np.sqrt(new_df["vx"]**2 + new_df["vy"]**2)
    new_df["a"] = np.sqrt(new_df["ax"]**2 + new_df["ay"]**2)
    new_df["jerk"] = np.sqrt(new_df["jx"]**2 + new_df["jy"]**2)

    # ------------------------------------------------------------------
    # Log magnitudes : preserves order and compresses large values
    # ------------------------------------------------------------------
    new_df["log_v"] = np.log1p(new_df["v"])    
    new_df["log_a"] = np.log1p(new_df["a"])
    new_df["log_j"] = np.log1p(new_df["jerk"])

    # ------------------------------------------------------------------
    # Directional features
    # ------------------------------------------------------------------
    new_df["angle"] = np.arctan2(new_df["vy"], new_df["vx"])
    new_df["delta_angle"] = np.gradient(new_df["angle"])

    # ------------------------------------------------------------------
    # Centered rolling statistics (smoothed)
    # ------------------------------------------------------------------
    new_df["v_mean"] = new_df["v"].rolling(smooth_window, center=True, min_periods=1).mean()
    new_df["v_std"]  = new_df["v"].rolling(smooth_window, center=True, min_periods=1).std().fillna(0)

    new_df["a_mean"] = new_df["a"].rolling(smooth_window, center=True, min_periods=1).mean()
    new_df["a_std"]  = new_df["a"].rolling(smooth_window, center=True, min_periods=1).std().fillna(0)

    new_df["j_mean"] = new_df["jerk"].rolling(smooth_window, center=True, min_periods=1).mean()
    new_df["j_std"]  = new_df["jerk"].rolling(smooth_window, center=True, min_periods=1).std().fillna(0)

    # ------------------------------------------------------------------
    # Motion sign changes
    # ------------------------------------------------------------------
    new_df["vx_sign"] = np.sign(new_df["vx"]).fillna(0.0)
    new_df["vx_sign_change"] = (
        new_df["vx_sign"].diff().abs() > 0
    ).astype(int)
    
    new_df["vy_sign"] = np.sign(new_df["vy"]).fillna(0.0)
    new_df["vy_sign_change"] = (
        new_df["vy_sign"].diff().abs() > 0
    ).astype(int)

    return new_df

# Select features
FEATURE_COLS = [
    "delta_angle",
    "vx_sign_change", 
    "vy_sign_change",
    "v", "a", "jerk",
    "vx", 'vy', 'ax', 'ay', 'jx', 'jy',
    "v_mean", "v_std",
    "a_mean", "a_std",
    "j_mean", "j_std",
    "log_v", "log_a", "log_j",
    "vx_abs_raw", "vy_abs_raw",
    "ax_abs_raw", "ay_abs_raw",
    "jx_abs_raw", "jy_abs_raw",
]
FEATURE_COLS_DEEP = FEATURE_COLS + ["x_i", "y_i"]
SMOOTH_WINDOW = 7

# Selecting df
df_copy = df.copy()
df_copy.index = pd.to_numeric(df_copy.index, errors="coerce")
df_copy = df_copy.sort_index()

# Train-Test Split
split_point = int(0.8 * len(df_copy))
train_df_raw = df_copy.iloc[:split_point]
test_df_raw  = df_copy.iloc[split_point:]

# DataFrames with processed features
train_df = build_features(train_df_raw, smooth_window=SMOOTH_WINDOW)
test_df  = build_features(test_df_raw,  smooth_window=SMOOTH_WINDOW)
X_train = train_df[FEATURE_COLS]
X_test  = test_df[FEATURE_COLS]
X_train_deep = train_df[FEATURE_COLS_DEEP]
X_test_deep  = test_df[FEATURE_COLS_DEEP]
y_train = train_df["action"].to_numpy()
y_test  = test_df["action"].to_numpy()

# Scaling (fit on train, apply to test)
scaler = StandardScaler()
scaler_deep = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)
X_train_deep_scaled = scaler_deep.fit_transform(X_train_deep)
X_test_deep_scaled  = scaler_deep.transform(X_test_deep)

# Encode labels
le = LabelEncoder()
y_train_labeled = le.fit_transform(y_train)
y_test_labeled  = le.transform(y_test)
classes = le.classes_
num_classes = len(classes)

# Saving preprocessors
preprocessors = {
    "scaler": scaler,
    "scaler_deep": scaler_deep,
    "label_encoder": le
}
joblib.dump(preprocessors, "preprocessors.joblib")

['preprocessors.joblib']

# Evaluation Function

In [9]:
def evaluation_with_and_without_tolerance(y_true, y_pred, tolerance=2, use_labels=True):
    """
    Event-level evaluation for temporal predictions with +/- tolerance.

    """

    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    if use_labels:
        event_classes = ["bounce", "hit"]
    else:
        event_classes = [1, 2]

    print(f"\nTemporal event evaluation (+/- {tolerance} frames)")
    print("-" * 52)
    print(f"{'Event':<12} | {'Precision':<10} | {'Recall':<10} | {'F1-Score':<10}")
    print("-" * 52)

    for event in event_classes:
        true_indices = np.where(y_true == event)[0]
        pred_indices = np.where(y_pred == event)[0]

        # ---------- Recall ----------
        matched_true = np.array([np.any(np.abs(pred_indices - t) <= tolerance) for t in true_indices])
        recall = matched_true.sum() / len(true_indices) if len(true_indices) > 0 else 0.0

        # ---------- Precision ----------
        matched_pred = np.array([np.any(np.abs(true_indices - p) <= tolerance) for p in pred_indices])
        precision = matched_pred.sum() / len(pred_indices) if len(pred_indices) > 0 else 0.0

        # ---------- F1 ----------
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

        event_name = str(event).capitalize() if use_labels else f"Class {event}"
        print(f"{event_name:<12} | {precision:>10.3f} | {recall:>10.3f} | {f1:>10.3f}")

# Unsupervised

In [364]:
# ==============================
# Compute adaptive thresholds
# ==============================
def calculate_thresholds(df_train):
    """
    Compute dynamic thresholds from training data for event detection.
    """
    thresholds = {
        "VERT_ACC_MIN": np.percentile(df_train["ay_abs_raw"].values, 70),
        "VERT_ACC_PROM": np.percentile(df_train["ay_abs_raw"].values, 85),
        "HORIZ_SPEED_DELTA": np.percentile(np.abs(np.diff(df_train["vx"].values)), 90),
        "HORIZ_SPEED_MIN": np.percentile(df_train["vx_abs_raw"].values, 10),
        "VERT_SPEED_MIN": np.percentile(df_train["vy_abs_raw"].values, 10),
        "VERT_ACC_BOUNCE": np.percentile(df_train["ay_raw"].values, 7),
        "JERK_THRESHOLD": np.percentile(df_train["jerk"].values, 90)
    }
    return thresholds

# ==============================
# Physics-based event detection
# ==============================
def detect_hits_and_bounces(df_test, thresholds, min_frames=10, window=2):
    """
    Detect events ("hit" or "bounce") from motion trajectory data.
    """
    df = df_test.copy()

    # Raw signals
    vert_acc = df["ay_raw"].values
    vert_acc_abs = df["ay_abs_raw"].values
    horiz_speed = df["vx_raw"].values
    vert_speed = df["vy_raw"].values
    jerk_vals = df["jerk"].values

    # Candidate event peaks
    peaks, _ = find_peaks(
        vert_acc_abs,
        height=thresholds["VERT_ACC_MIN"],
        prominence=thresholds["VERT_ACC_PROM"],
        distance=3
    )

    candidate_events = []

    for idx in peaks:
        if idx < window or idx + window >= len(df):
            continue

        # Velocity states around event
        vx_before, vx_after = horiz_speed[idx - window], horiz_speed[idx + window]
        vy_before, vy_after = vert_speed[idx - window], vert_speed[idx + window]
        ay_val = vert_acc[idx]
        jerk_val = jerk_vals[idx]

        # Derived physics metrics
        horiz_flip = vx_before * vx_after < 0
        vert_flip = vy_before * vy_after < 0
        horiz_delta = abs(vx_after) - abs(vx_before)
        vert_ratio = abs(vy_after) / (abs(vy_before) + 1e-6)
        angle_before = np.arctan2(vy_before, vx_before)
        angle_after = np.arctan2(vy_after, vx_after)
        angle_change = abs(angle_after - angle_before)

        # Event scoring
        hit_points = 0.0
        bounce_points = 0.0

        # Hit scoring
        hit_points += 2.0 if horiz_flip else 0.0 # vx changes direction
        hit_points += 1.5 if horiz_delta > thresholds["HORIZ_SPEED_DELTA"] else 0.0 # Magnitude of vx increases sharply in additional speed (not in ratio)
        hit_points += 1.0 if vert_ratio > 1.1 else 0.0 # vy after the event increases by more than 10% (in magnitude)
        hit_points += 1.0 if jerk_val > thresholds["JERK_THRESHOLD"] else 0.0 # If the rate of change of acceleration is high
        hit_points += 1.0 if angle_change > np.pi / 4 else 0.0 # If trajectory angle changes by more than 45°

        # Bounce scoring
        bounce_points += 2.0 if ay_val < thresholds["VERT_ACC_BOUNCE"] else 0.0 # ay is very negative
        bounce_points += 1.5 if vert_flip else 0.0 # vy changes direction
        bounce_points += 1.0 if vert_ratio < 0.8 else 0.0 # vy after the event decrease by more than 20% (energy loss)

        # Decide event type
        event_type = None
        if hit_points >= bounce_points and hit_points >= 2.5:
            event_type = "hit"
        elif bounce_points > hit_points and bounce_points >= 2.0:
            event_type = "bounce"

        # Computing strength score to help chosing between close events
        strength_score = abs(ay_val) + jerk_val + abs(horiz_delta) + angle_change

        if event_type:
            candidate_events.append((df.index[idx], event_type, strength_score))

    # ==============================
    # Strongest Event Selection (to prevent from selecting event too close)
    # ==============================
    candidate_events.sort(key=lambda x: x[0])
    final_events = {}

    for frame_id, label, score in candidate_events:
        if not final_events:
            final_events[frame_id] = (label, score)
            continue

        last_frame = max(final_events.keys())
        if frame_id - last_frame >= min_frames:
            final_events[frame_id] = (label, score)
        else:
            # Keep the event with higher strength
            if score > final_events[last_frame][1]:
                final_events[last_frame] = (label, score)

    # Return events by frame
    return {frame: label for frame, (label, _) in final_events.items()}

In [366]:
# Evluation 
thresh = calculate_thresholds(train_df)
joblib.dump(thresh, "thresholds_physics.joblib")
y_pred = detect_hits_and_bounces(test_df, thresholds=thresh)
y_pred_array = np.array(["air"] * len(y_test), dtype=object)
for frame, label in y_pred.items():
    if frame in test_df.index:
        idx = test_df.index.get_loc(frame)
        y_pred_array[idx] = label

print("="*18 +"Standard Evaluation"+"="*18)
print(classification_report(
    y_test,
    y_pred_array,
    target_names=classes,
    zero_division=0
))
print(confusion_matrix(y_test, y_pred_array))

print("="*13 +"Temporal Tolerance Evaluation"+"="*13)
evaluation_with_and_without_tolerance(
    y_true=y_test,
    y_pred=y_pred_array,
    use_labels=True
)

==================Standard Evaluation==================
              precision    recall  f1-score   support

         air       0.98      0.98      0.98     23385
      bounce       0.24      0.32      0.27       308
         hit       0.41      0.35      0.38       323

    accuracy                           0.96     24016
   macro avg       0.54      0.55      0.54     24016
weighted avg       0.97      0.96      0.96     24016

[[22928   304   153]
 [  199   100     9]
 [  195    16   112]]
=============Temporal Tolerance Evaluation=============

Temporal event evaluation (+/- 2 frames)
----------------------------------------------------
Event        | Precision  | Recall     | F1-Score  
----------------------------------------------------
Bounce       |      0.507 |      0.692 |      0.585
Hit          |      0.635 |      0.539 |      0.583


# Supervised

## 1.1 Random Forest

In [ ]:
# Baseline model
rf = RandomForestClassifier(
    n_estimators=400,
    class_weight="balanced",   # to help with class imbalance
    random_state=42,
    n_jobs=-1
)

# Time-aware CV to preserve order of the frames and a gap to avoid data leakage
tscv = TimeSeriesSplit(n_splits=5, gap=SMOOTH_WINDOW // 2 + 1)

param_grid = {
    "max_depth": [40, 50, 60],
    "min_samples_split": [6, 7],
    "min_samples_leaf": [2,3,4],
    "max_features": ["sqrt","log2", None]
}

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=tscv,
    scoring="f1_macro", # Each class’s F1 contributes equally, to help with class imbalance
    n_jobs=-1,
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
best_rf = grid.best_estimator_

y_pred = best_rf.predict(X_test)

print("="*18 +"Standard Evaluation"+"="*18)
print(classification_report(
    y_test,
    y_pred,
    target_names=classes,
    zero_division=0
))
print(confusion_matrix(y_test, y_pred))

print("="*13 +"Temporal Tolerance Evaluation"+"="*13)
evaluation_with_and_without_tolerance(
    y_test,
    y_pred,
    tolerance=2,
    use_labels=True
)

joblib.dump(grid, "model/rf_model.joblib")

KeyboardInterrupt: 

## 1.2 Balanced Random Forest

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# 1) Time-aware CV with a small gap to avoid centered-window bleed
tscv = TimeSeriesSplit(n_splits=5, gap=SMOOTH_WINDOW // 2 + 1)

# 2) Balanced RF (undersampling per tree)
rf = BalancedRandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

# 3) Lean grid
param_grid = {
    "max_depth": [None, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt", "log2", None],
}

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=tscv,
    scoring="precision",
    n_jobs=-1,
    refit=True,
)

grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)

best_rf = grid.best_estimator_
y_pred = best_rf.predict(X_test)

print("="*18 +"Standard Evaluation"+"="*18)
print(classification_report(
    y_test,
    y_pred,
    target_names=classes,
    zero_division=0
))
print(confusion_matrix(y_test_labeled, y_pred))

print("="*13 +"Temporal Tolerance Evaluation"+"="*13)
evaluation_with_and_without_tolerance(
    y_test,
    y_pred,
    tolerance=2,
    use_labels=True
)

joblib.dump(grid, "unused_models/rfus_model.joblib")


## 2.1 XG BOOST

In [ ]:
# Weights per class
sample_weights = compute_sample_weight(
    class_weight="balanced",
    y=y_train_labeled
)

# Boost non-zero classes (hits / bounces)
sample_weights[y_train_labeled > 0] *= 5

f05_scorer = make_scorer(
    fbeta_score,
    beta=0.5,
    average="macro"
)

xgb = XGBClassifier(
    objective="multi:softprob",
    num_class=len(classes),
    tree_method="hist",
    eval_metric="mlogloss",
    n_estimators=400,
    random_state=42,
    n_jobs=-1
)

tscv = TimeSeriesSplit(
    n_splits=5,
    gap=SMOOTH_WINDOW // 2 + 1
)

param_grid = {
    "max_depth": [3, 6],
    "learning_rate": [0.03, 0.07],
    "subsample": [0.7, 1.0],
    "colsample_bytree": [0.7, 1.0],
    "min_child_weight": [1, 5],
    "gamma": [0.0, 1.0],
}

grid = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=tscv,
    scoring=f05_scorer,
    n_jobs=-1,
    refit=True,
    verbose=1
)

grid.fit(
    X_train,
    y_train_labeled,
    sample_weight=sample_weights
)

print("Best parameters:", grid.best_params_)
best_xgb = grid.best_estimator_


y_pred = best_xgb.predict(X_test)

print("="*18 +"Standard Evaluation"+"="*18)
print(classification_report(
    y_test_labeled,
    y_pred,
    target_names=classes,
    zero_division=0
))
print(confusion_matrix(y_test_labeled, y_pred))

print("="*13 +"Temporal Tolerance Evaluation"+"="*13)
evaluation_with_and_without_tolerance(
    y_test_labeled,
    y_pred,
    tolerance=2,
    use_labels=False
)

joblib.dump(grid, "unused_models/xgb_model.joblib")

## 2.2 XG BOOST with Undersampling

In [ ]:
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

pipe = ImbPipeline(steps=[
    ("rus", RandomUnderSampler(random_state=42)),  # undersampling de la majority class
    ("xgb", XGBClassifier(
        objective="multi:softprob",
        num_class=len(np.unique(y_train)),
        tree_method="hist",
        n_estimators=300,
        random_state=42,
        n_jobs=-1,
        eval_metric="mlogloss",
    ))
])


tscv = TimeSeriesSplit(n_splits=5, gap=SMOOTH_WINDOW // 2 + 1)

param_grid = {
    "xgb__max_depth": [3, 6],
    "xgb__learning_rate": [0.05, 0.1],
    "xgb__subsample": [0.7, 1.0],
    "xgb__colsample_bytree": [0.7, 1.0],
    "xgb__min_child_weight": [1, 5],
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=tscv,
    scoring="f1_macro",
    n_jobs=-1,
    refit=True,
)

grid.fit(X_train, y_train_labeled)
print("Best params:", grid.best_params_)

best_pipe = grid.best_estimator_
y_pred = best_pipe.predict(X_test)

print("="*18 +"Standard Evaluation"+"="*18)
print(classification_report(
    y_test_labeled,
    y_pred,
    target_names=classes,
    zero_division=0
))
print(confusion_matrix(y_test_labeled, y_pred))

print("="*13 +"Temporal Tolerance Evaluation"+"="*13)
evaluation_with_and_without_tolerance(
    y_test_labeled,
    y_pred,
    tolerance=2,
    use_labels=False
)

joblib.dump(grid, "unused_models/xgbus_model.joblib")

## 3. MLP

In [ ]:
# ====== Make a small validation split from the tail of train (chronological) ======
val_ratio = 0.1
split_idx = int((1.0 - val_ratio) * len(X_train_scaled))
X_train_mlp, X_val_mlp = X_train_scaled[:split_idx], X_train_scaled[split_idx:]
y_train_mlp, y_val_mlp = y_train_labeled[:split_idx], y_train_labeled[split_idx:]


# --- Your model builder ---
def build_mlp(input_dim, num_classes):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=[]  
    )
    return model

mlp = build_mlp(X_train_mlp.shape[1], num_classes)

# --- Macro F1 callback ---
class MacroF1Callback(keras.callbacks.Callback):
    def __init__(self, X_val, y_val, patience=5):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = -np.inf
        self.best_weights = None
        self.patience = patience
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        y_proba = self.model.predict(self.X_val, verbose=0)
        y_pred = y_proba.argmax(axis=1)
        f1_macro = f1_score(self.y_val, y_pred, average="macro", zero_division=0)
        logs = logs or {}
        logs["val_f1_macro"] = f1_macro
        print(f" — val_f1_macro: {f1_macro:.4f}")

        if f1_macro > self.best_f1:
            self.best_f1 = f1_macro
            self.best_weights = self.model.get_weights()
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"Early stopping on macro F1 (patience={self.patience}). Restoring best weights.")
                self.model.stop_training = True
                if self.best_weights is not None:
                    self.model.set_weights(self.best_weights)

macro_f1_cb = MacroF1Callback(X_val_mlp, y_val_mlp, patience=5)

# --- Other callbacks for stability ---
callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5),
    macro_f1_cb,
]

# Weights per class
sample_weights_mlp = compute_sample_weight(
    class_weight="balanced",
    y=y_train_mlp
)
# Boost non-zero classes (hits / bounces)
sample_weights_mlp[y_train_mlp > 0] *= 5

history = mlp.fit(
    X_train_mlp, y_train_mlp,
    validation_data=(X_val_mlp, y_val_mlp),
    epochs=30,
    batch_size=128,
    sample_weight=sample_weights_mlp,
    callbacks=callbacks,
)


# ====== Evaluate on test ======
y_proba = mlp.predict(X_test_scaled, batch_size=256)
y_pred  = y_proba.argmax(axis=1)

print("="*18 +"Standard Evaluation"+"="*18)
print(classification_report(
    y_test_labeled,
    y_pred,
    target_names=classes,
    zero_division=0
))
print(confusion_matrix(y_test_labeled, y_pred))

print("="*13 +"Temporal Tolerance Evaluation"+"="*13)
evaluation_with_and_without_tolerance(
    y_test_labeled,
    y_pred,
    tolerance=2,
    use_labels=False
)

joblib.dump(mlp, "unused_models/mlp_model.joblib")


## 4. LSTM

In [ ]:
# ----- Macro F1 callback -----
class MacroF1Callback(keras.callbacks.Callback):
    def __init__(self, X_val, y_val, patience=6):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = -np.inf
        self.best_weights = None
        self.patience = patience
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        y_proba = self.model.predict(self.X_val, verbose=0, batch_size=128)
        y_pred = y_proba.argmax(axis=1)
        f1_macro = f1_score(self.y_val, y_pred, average="macro", zero_division=0)
        logs = logs or {}
        logs["val_f1_macro"] = f1_macro
        print(f" — val_f1_macro: {f1_macro:.4f}")

        if f1_macro > self.best_f1:
            self.best_f1 = f1_macro
            self.best_weights = self.model.get_weights()
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"Early stopping on macro F1 (patience={self.patience}). Restoring best weights.")
                self.model.stop_training = True
                if self.best_weights is not None:
                    self.model.set_weights(self.best_weights)


def build_lstm(window_size, feature_dim, num_classes, bidirectional=False):
    inputs = keras.Input(shape=(window_size, feature_dim))

    x = inputs
    if bidirectional:
        x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
        x = layers.Bidirectional(layers.LSTM(64))(x)
    else:
        x = layers.LSTM(64, return_sequences=True)(x)
        x = layers.LSTM(64)(x)

    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=[]  
    )
    return model

# ====== Sequences (from your code) ======
def make_sequences(X: np.ndarray, y: np.ndarray, window: int = 7, stride: int = 1):
    X_seq, y_seq = [], []
    for start in range(0, len(X) - window + 1, stride):
        end = start + window
        X_seq.append(X[start:end])
        mid_idx = start + window // 2
        y_seq.append(y[mid_idx])
    return np.array(X_seq), np.array(y_seq)

window = 7
stride = 1

X_train_seq, y_train_seq = make_sequences(X_train_deep_scaled, y_train_labeled, window=window, stride=stride)
X_test_seq,  y_test_seq  = make_sequences(X_test_deep_scaled,  y_test_labeled,  window=window, stride=stride)

num_classes = len(classes)
feature_dim = X_train_seq.shape[-1]

# Chronological validation split (tail)
val_ratio = 0.1
split_idx = int((1.0 - val_ratio) * len(X_train_seq))
X_train_lstm, X_val_lstm = X_train_seq[:split_idx], X_train_seq[split_idx:]
y_train_lstm, y_val_lstm = y_train_seq[:split_idx], y_train_seq[split_idx:]

# Weights per class
sample_weights_lstm = compute_sample_weight(
    class_weight="balanced",
    y=y_train_lstm
)
# Boost non-zero classes (hits / bounces)
sample_weights_lstm[y_train_lstm > 0] *= 5

# ====== Build LSTM ======
lstm = build_lstm(window, feature_dim, num_classes, bidirectional=True)

# Callbacks: LR on val_loss, early stop/restore on val macro F1 via custom callback
callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor="val_f1_macro", mode="max", factor=0.5, patience=3, min_lr=1e-5),
    MacroF1Callback(X_val_lstm, y_val_lstm, patience=6),
]

# ====== Train ======
history = lstm.fit(
    X_train_lstm, y_train_lstm,
    validation_data=(X_val_lstm, y_val_lstm),
    epochs=40,
    batch_size=128,
    sample_weight=sample_weights_lstm,
    callbacks=callbacks,
)


# ====== Evaluate ======
y_proba_seq = lstm.predict(X_test_seq, batch_size=128)
y_pred_seq  = y_proba_seq.argmax(axis=1)

print("="*18 +"Standard Evaluation"+"="*18)
print(classification_report(
    y_test_seq,
    y_pred_seq,
    target_names=classes,
    zero_division=0
))
print(confusion_matrix(y_test_seq, y_pred_seq))

print("="*13 +"Temporal Tolerance Evaluation"+"="*13)
evaluation_with_and_without_tolerance(
    y_test_seq,
    y_pred_seq,
    tolerance=2,
    use_labels=False
)

joblib.dump(lstm, "unused_models/lstm_model.joblib")


In [3]:
from main import unsupervized_hit_bounce_detection, supervized_hit_bounce_detection

supervized_hit_bounce_detection(json_dir/"ball_data_320.json")

Predictions added to '/home/andreasab/git-folder/hit-bounce-detection/per_point_v2/ball_data_320.json' successfully!


{865944: {'x': 822, 'y': 278, 'visible': True, 'action': 'air'},
 865972: {'x': 880.0729166666667,
  'y': 168.30208333333348,
  'visible': True,
  'action': 'air'},
 865974: {'x': 885, 'y': 197, 'visible': True, 'action': 'air'},
 865975: {'x': 889, 'y': 213, 'visible': True, 'action': 'air'},
 865976: {'x': 891, 'y': 224, 'visible': True, 'action': 'air'},
 865977: {'x': 892, 'y': 235, 'visible': True, 'action': 'air'},
 865978: {'x': 893, 'y': 247, 'visible': True, 'action': 'air'},
 865979: {'x': 895, 'y': 263, 'visible': True, 'action': 'air'},
 865984: {'x': 914, 'y': 341, 'visible': True, 'action': 'air'},
 865985: {'x': 818, 'y': 277, 'visible': True, 'action': 'air'},
 865986: {'x': 922, 'y': 377, 'visible': True, 'action': 'air'},
 865987: {'x': 928, 'y': 393, 'visible': True, 'action': 'air'},
 865988: {'x': 933, 'y': 414, 'visible': True, 'action': 'air'},
 865989: {'x': 938, 'y': 435, 'visible': True, 'action': 'air'},
 865990: {'x': 943, 'y': 453, 'visible': True, 'action'